In [1]:
import pandas as pd
import datetime
import numpy as np
from joblib import dump, load

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

# Load the Data

In [2]:
df = pd.read_csv("./RAW_DATA/dublinbus_jan.csv")

### Information on the data

Print the number of rows and columns for the data

In [3]:
print("Rows:" ,df.shape[0])
print("Columns:" ,df.shape[1])

Rows: 50000
Columns: 16


Print the head and tail of the data to examine it in more detail

In [4]:
df.head()

,DayOfService,TripID,LineId,Direction,TPlannedTime_Arr,TPlannedTime_Dep,TActualTime_Arr,TActualTime_Dep,progrnumber,stoppointid,Lplannedtime_arr,Lplannedtime_dep,Lactualtime_arr,Lactualtime_dep,vehicleid,lastupdate
0,2018-01-01 00:00:00,5955277,16,1,33932,30000,33184.0,30029.0,28,45,31343,31343,31152,31163,1001127,2018-01-08 17:21:10
1,2018-01-01 00:00:00,5955277,16,1,33932,30000,33184.0,30029.0,29,46,31407,31407,31207,31239,1001127,2018-01-08 17:21:10
2,2018-01-01 00:00:00,5955277,16,1,33932,30000,33184.0,30029.0,30,47,31484,31484,31281,31288,1001127,2018-01-08 17:21:10
3,2018-01-01 00:00:00,5955277,16,1,33932,30000,33184.0,30029.0,31,48,31551,31551,31315,31327,1001127,2018-01-08 17:21:10
4,2018-01-01 00:00:00,5955277,16,1,33932,30000,33184.0,30029.0,32,49,31646,31646,31362,31439,1001127,2018-01-08 17:21:10


In [5]:
df.tail()

,DayOfService,TripID,LineId,Direction,TPlannedTime_Arr,TPlannedTime_Dep,TActualTime_Arr,TActualTime_Dep,progrnumber,stoppointid,Lplannedtime_arr,Lplannedtime_dep,Lactualtime_arr,Lactualtime_dep,vehicleid,lastupdate
49995,2018-01-01 00:00:00,5959767,83,2,69259,65400,68674.0,65469.0,21,1071,66293,66293,66354,66354,2693245,2018-01-08 17:21:10
49996,2018-01-01 00:00:00,5959767,83,2,69259,65400,68674.0,65469.0,22,4528,66348,66348,66382,66382,2693245,2018-01-08 17:21:10
49997,2018-01-01 00:00:00,5959767,83,2,69259,65400,68674.0,65469.0,23,1072,66424,66424,66467,66478,2693245,2018-01-08 17:21:10
49998,2018-01-01 00:00:00,5959767,83,2,69259,65400,68674.0,65469.0,24,1352,66542,66542,66581,66620,2693245,2018-01-08 17:21:10
49999,2018-01-01 00:00:00,5959767,83,2,69259,65400,68674.0,65469.0,25,1353,66596,66596,66645,66645,2693245,2018-01-08 17:21:10


### Examine the data types

In [6]:
df.dtypes

DayOfService         object
TripID                int64
LineId               object
Direction             int64
TPlannedTime_Arr      int64
TPlannedTime_Dep      int64
TActualTime_Arr     float64
TActualTime_Dep     float64
progrnumber           int64
stoppointid           int64
Lplannedtime_arr      int64
Lplannedtime_dep      int64
Lactualtime_arr       int64
Lactualtime_dep       int64
vehicleid             int64
lastupdate           object
dtype: object

### Reassign the datatypes to suitable forms

In [7]:
categorical_columns = df[['LineId']].columns
continuous_columns = df[['TripID','Direction','TPlannedTime_Arr','TPlannedTime_Dep','TActualTime_Arr','TActualTime_Dep','progrnumber','stoppointid','Lplannedtime_arr','Lplannedtime_dep','Lactualtime_arr','Lactualtime_dep','vehicleid']].columns
datetime_columns = df[['DayOfService', 'lastupdate']]

for column in continuous_columns:
    df[column] = df[column].astype('float64')  

for column in categorical_columns:
    df[column] = df[column].astype('category')  

for column in datetime_columns:
    df[column] = pd.to_datetime(df[column]).astype('int64')  
    
df.dtypes

DayOfService           int64
TripID               float64
LineId              category
Direction            float64
TPlannedTime_Arr     float64
TPlannedTime_Dep     float64
TActualTime_Arr      float64
TActualTime_Dep      float64
progrnumber          float64
stoppointid          float64
Lplannedtime_arr     float64
Lplannedtime_dep     float64
Lactualtime_arr      float64
Lactualtime_dep      float64
vehicleid            float64
lastupdate             int64
dtype: object

# Initial Checks on Data

### Examine Duplicate columns

In [8]:
dfT = df.T
print("Number of duplicate (excluding first) columns in the table is: ", dfT.duplicated().sum())
print("Number of duplicate (including first) columns in the table is: ",  dfT[dfT.duplicated(keep=False)].shape[0])

Number of duplicate (excluding first) columns in the table is:  1
Number of duplicate (including first) columns in the table is:  2


### Examine the columns in more detail 

In [9]:
df.select_dtypes(['category']).describe().T

,count,unique,top,freq
LineId,50000,68,9,2378


In [10]:
df.select_dtypes(['int64']).describe().T

,count,mean,std,min,25%,50%,75%,max
DayOfService,50000.0,1.514765e+18,0.00000,1.514765e+18,1.514765e+18,1.514765e+18,1.514765e+18,1.514765e+18
lastupdate,50000.0,1.515432e+18,768.00768,1.515432e+18,1.515432e+18,1.515432e+18,1.515432e+18,1.515432e+18


In [11]:
df.select_dtypes(['float64']).describe().T

,count,mean,std,min,25%,50%,75%,max
TripID,50000.0,5.957371e+06,1306.330204,5955277.0,5956288.00,5957473.0,5958429.00,5959767.0
Direction,50000.0,1.506080e+00,0.499968,1.0,1.00,2.0,2.00,2.0
TPlannedTime_Arr,50000.0,6.116368e+04,14876.462373,26496.0,48720.00,61288.0,73623.00,88603.0
TPlannedTime_Dep,50000.0,5.768153e+04,15028.899527,23700.0,45000.00,57600.0,70200.00,85500.0
TActualTime_Arr,47903.0,6.054521e+04,14882.724619,26447.0,48087.00,60543.0,73026.00,87916.0
TActualTime_Dep,47814.0,5.798451e+04,14899.964928,25517.0,45179.00,57620.0,70207.00,85501.0
progrnumber,50000.0,3.092208e+01,19.887865,1.0,14.00,29.0,45.00,102.0
stoppointid,50000.0,2.517442e+03,2042.708130,2.0,830.00,2017.5,3885.00,7665.0
Lplannedtime_arr,50000.0,5.941817e+04,14976.874464,23799.0,47067.75,59469.0,71826.25,88603.0
Lplannedtime_dep,50000.0,5.941817e+04,14976.874464,23799.0,47067.75,59469.0,71826.25,88603.0


# Examine duplicate rows in the data

In [12]:
print('Number of duplicate (excluding first) rows in the table is: ', df.duplicated().sum())
print('Number of duplicate rows (including first) in the table is:', df[df.duplicated(keep=False)].shape[0])

Number of duplicate (excluding first) rows in the table is:  0
Number of duplicate rows (including first) in the table is: 0


### Print out the duplicate rows

In [13]:
df[df.duplicated(keep=False)].sort_values(by = df.columns.to_list())

,DayOfService,TripID,LineId,Direction,TPlannedTime_Arr,TPlannedTime_Dep,TActualTime_Arr,TActualTime_Dep,progrnumber,stoppointid,Lplannedtime_arr,Lplannedtime_dep,Lactualtime_arr,Lactualtime_dep,vehicleid,lastupdate


# Analyse the data for missing values

In [14]:
df.isnull().sum()

DayOfService           0
TripID                 0
LineId                 0
Direction              0
TPlannedTime_Arr       0
TPlannedTime_Dep       0
TActualTime_Arr     2097
TActualTime_Dep     2186
progrnumber            0
stoppointid            0
Lplannedtime_arr       0
Lplannedtime_dep       0
Lactualtime_arr        0
Lactualtime_dep        0
vehicleid              0
lastupdate             0
dtype: int64

# Findings

    - Duplicate cols
        - drop one of the duplicate cols
    - Null values
        - drop rows with actual time arrival missing as this is the target outcome, inpute/ create null category for actual time depart

In [15]:
arr_null = df[df["TActualTime_Arr"].isnull()].index

In [16]:
arr_null

Int64Index([ 1394,  1395,  1396,  1397,  1398,  1399,  1400,  1401,  1402,
             1403,
            ...
            49481, 49482, 49483, 49484, 49485, 49486, 49487, 49488, 49489,
            49490],
           dtype='int64', length=2097)

In [17]:
df = df.drop(arr_null)

In [18]:
df.shape

(47903, 16)

The best guess that we have for the actual time of departue is the planned time of departure

In [19]:
df["TActualTime_Dep"].fillna(df["TPlannedTime_Dep"], inplace=True)

In [20]:
df.isnull().sum()

DayOfService        0
TripID              0
LineId              0
Direction           0
TPlannedTime_Arr    0
TPlannedTime_Dep    0
TActualTime_Arr     0
TActualTime_Dep     0
progrnumber         0
stoppointid         0
Lplannedtime_arr    0
Lplannedtime_dep    0
Lactualtime_arr     0
Lactualtime_dep     0
vehicleid           0
lastupdate          0
dtype: int64

# Weather Data

### Import the weather data

In [21]:
wdf = pd.read_csv("./RAW_DATA/weather_clean.csv")

### Examine the weather data

In [22]:
wdf.head()

,weatherdate,rain,temp,wetb,dewpt,vappr,rhum,msl
0,2018-01-01 00:00:00,0.0,4.6,3.5,1.8,6.9,82.0,991.0
1,2018-01-01 01:00:00,0.1,4.7,3.6,1.8,7.0,81.0,991.1
2,2018-01-01 02:00:00,0.0,4.8,3.7,1.9,7.0,81.0,991.1
3,2018-01-01 03:00:00,0.0,4.9,3.8,2.2,7.2,82.0,990.7
4,2018-01-01 04:00:00,0.0,5.3,4.1,2.3,7.2,81.0,990.3


In [23]:
wdf.dtypes

weatherdate     object
rain           float64
temp           float64
wetb           float64
dewpt          float64
vappr          float64
rhum           float64
msl            float64
dtype: object

In [24]:
wdf["weatherdate"] = pd.to_datetime(wdf["weatherdate"]).astype('datetime64[ns]') 

In [25]:
wdf.head()

,weatherdate,rain,temp,wetb,dewpt,vappr,rhum,msl
0,2018-01-01 00:00:00,0.0,4.6,3.5,1.8,6.9,82.0,991.0
1,2018-01-01 01:00:00,0.1,4.7,3.6,1.8,7.0,81.0,991.1
2,2018-01-01 02:00:00,0.0,4.8,3.7,1.9,7.0,81.0,991.1
3,2018-01-01 03:00:00,0.0,4.9,3.8,2.2,7.2,82.0,990.7
4,2018-01-01 04:00:00,0.0,5.3,4.1,2.3,7.2,81.0,990.3


# Merge Weather and Dublin Bus data

Create a shared column for actual arrival time, determine the weather for when the bus is planned to arrive to determine the weather in dublin at that time the bus is travelling

In [26]:
df["weatherdate"] = pd.to_datetime(df["DayOfService"]) + pd.to_timedelta(df["Lplannedtime_arr"], unit='s')
df['weatherdate'] = df['weatherdate'].apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day, dt.hour))

In [27]:
df.head()

,DayOfService,TripID,LineId,Direction,TPlannedTime_Arr,TPlannedTime_Dep,TActualTime_Arr,TActualTime_Dep,progrnumber,stoppointid,Lplannedtime_arr,Lplannedtime_dep,Lactualtime_arr,Lactualtime_dep,vehicleid,lastupdate,weatherdate
0,1514764800000000000,5955277.0,16,1.0,33932.0,30000.0,33184.0,30029.0,28.0,45.0,31343.0,31343.0,31152.0,31163.0,1001127.0,1515432070000000000,2018-01-01 08:00:00
1,1514764800000000000,5955277.0,16,1.0,33932.0,30000.0,33184.0,30029.0,29.0,46.0,31407.0,31407.0,31207.0,31239.0,1001127.0,1515432070000000000,2018-01-01 08:00:00
2,1514764800000000000,5955277.0,16,1.0,33932.0,30000.0,33184.0,30029.0,30.0,47.0,31484.0,31484.0,31281.0,31288.0,1001127.0,1515432070000000000,2018-01-01 08:00:00
3,1514764800000000000,5955277.0,16,1.0,33932.0,30000.0,33184.0,30029.0,31.0,48.0,31551.0,31551.0,31315.0,31327.0,1001127.0,1515432070000000000,2018-01-01 08:00:00
4,1514764800000000000,5955277.0,16,1.0,33932.0,30000.0,33184.0,30029.0,32.0,49.0,31646.0,31646.0,31362.0,31439.0,1001127.0,1515432070000000000,2018-01-01 08:00:00


merge the weather and dublin bus on this new column

In [28]:
final = pd.merge(df,wdf, on='weatherdate')

In [29]:
final.columns

Index(['DayOfService', 'TripID', 'LineId', 'Direction', 'TPlannedTime_Arr',
       'TPlannedTime_Dep', 'TActualTime_Arr', 'TActualTime_Dep', 'progrnumber',
       'stoppointid', 'Lplannedtime_arr', 'Lplannedtime_dep',
       'Lactualtime_arr', 'Lactualtime_dep', 'vehicleid', 'lastupdate',
       'weatherdate', 'rain', 'temp', 'wetb', 'dewpt', 'vappr', 'rhum', 'msl'],
      dtype='object')

remove the weather date column as no longer necessary

In [30]:
final = final.drop('weatherdate', 1)

# Inspect the new dataframe

In [31]:
final.shape

(47903, 23)

In [32]:
final.head()

,DayOfService,TripID,LineId,Direction,TPlannedTime_Arr,TPlannedTime_Dep,TActualTime_Arr,TActualTime_Dep,progrnumber,stoppointid,...,Lactualtime_dep,vehicleid,lastupdate,rain,temp,wetb,dewpt,vappr,rhum,msl
0,1514764800000000000,5955277.0,16,1.0,33932.0,30000.0,33184.0,30029.0,28.0,45.0,...,31163.0,1001127.0,1515432070000000000,0.0,4.3,3.4,2.0,7.0,84.0,991.8
1,1514764800000000000,5955277.0,16,1.0,33932.0,30000.0,33184.0,30029.0,29.0,46.0,...,31239.0,1001127.0,1515432070000000000,0.0,4.3,3.4,2.0,7.0,84.0,991.8
2,1514764800000000000,5955277.0,16,1.0,33932.0,30000.0,33184.0,30029.0,30.0,47.0,...,31288.0,1001127.0,1515432070000000000,0.0,4.3,3.4,2.0,7.0,84.0,991.8
3,1514764800000000000,5955277.0,16,1.0,33932.0,30000.0,33184.0,30029.0,31.0,48.0,...,31327.0,1001127.0,1515432070000000000,0.0,4.3,3.4,2.0,7.0,84.0,991.8
4,1514764800000000000,5955277.0,16,1.0,33932.0,30000.0,33184.0,30029.0,32.0,49.0,...,31439.0,1001127.0,1515432070000000000,0.0,4.3,3.4,2.0,7.0,84.0,991.8


# Prepare the datframe for the model

In [33]:
final = pd.get_dummies(final)

In [34]:
y = final["TActualTime_Arr"]

In [35]:
X_col = list(final.columns)
X_col.remove("TActualTime_Arr")

In [36]:
X = final[X_col]

In [37]:
test_size = int(X.shape[0] * 0.7)

In [38]:
X_train, X_test, y_train, y_test = X.loc[final.index[:test_size]], X.loc[final.index[test_size:]], y.loc[final.index[:test_size]], y.loc[final.index[test_size:]]

In [39]:
X_train.shape

(33532, 89)

In [40]:
X_train.columns

Index(['DayOfService', 'TripID', 'Direction', 'TPlannedTime_Arr',
       'TPlannedTime_Dep', 'TActualTime_Dep', 'progrnumber', 'stoppointid',
       'Lplannedtime_arr', 'Lplannedtime_dep', 'Lactualtime_arr',
       'Lactualtime_dep', 'vehicleid', 'lastupdate', 'rain', 'temp', 'wetb',
       'dewpt', 'vappr', 'rhum', 'msl', 'LineId_1', 'LineId_102', 'LineId_11',
       'LineId_120', 'LineId_123', 'LineId_13', 'LineId_130', 'LineId_14',
       'LineId_140', 'LineId_145', 'LineId_14C', 'LineId_15', 'LineId_151',
       'LineId_16', 'LineId_16C', 'LineId_17A', 'LineId_18', 'LineId_184',
       'LineId_220', 'LineId_25A', 'LineId_25B', 'LineId_26', 'LineId_27',
       'LineId_270', 'LineId_27A', 'LineId_27B', 'LineId_29A', 'LineId_31',
       'LineId_32', 'LineId_33', 'LineId_33A', 'LineId_33B', 'LineId_37',
       'LineId_38', 'LineId_38A', 'LineId_39', 'LineId_39A', 'LineId_4',
       'LineId_40', 'LineId_40B', 'LineId_40D', 'LineId_41', 'LineId_41B',
       'LineId_41C', 'LineId_42', 'Li

In [41]:
linreg = LinearRegression().fit(X_train, y_train)

In [42]:
results = linreg.predict(X_train)

In [43]:
print("MAE", metrics.mean_absolute_error(y_train, results))
print("MSE",metrics.mean_squared_error(y_train, results))
print("RMSE",np.sqrt(metrics.mean_squared_error(y_train, results)))

MAE 246.33910083691117
MSE 143086.3436030355
RMSE 378.267555578106


In [44]:
mean_time_error = sum(abs(y_train - results))/ len(y_train)

In [45]:
str(pd.to_timedelta(mean_time_error, unit='s'))

'0 days 00:04:06.339100837'

In [46]:
results = linreg.predict(X_test)

In [47]:
print("MAE", metrics.mean_absolute_error(y_test, results))
print("MSE",metrics.mean_squared_error(y_test, results))
print("RMSE",np.sqrt(metrics.mean_squared_error(y_test, results)))

MAE 292.7127973213963
MSE 170340.6258074801
RMSE 412.72342531952324


In [48]:
mean_time_error = sum(abs(y_test - results))/ len(y_test)
str(pd.to_timedelta(mean_time_error, unit='s'))

'0 days 00:04:52.712797321'

In [49]:
dump(linreg, "./MODELS/january.joblib")
model = load("./MODELS/january.joblib")
model.predict(X_test)

array([74412.30233765, 74467.31913757, 69231.24475098, ...,
       26409.07618713, 26419.20217896, 26413.68605042])

In [50]:
print("Success")

Success
